# Converting wavefunctions to matrix-product state (MPS) format

While the SOS format, implemented in Overlapper with the sparse dictionary, is convenient for many use cases, sometimes the MPS format is indispensible. In particular, this conversion is frequently necessary to approximate energy distributions.

Regardless of whether energy distributions are computed via the series expansion method or via the resolvent method, the MPS form is beneficial because it allows working with much larger systems. For the former, it enables the computation of moments; for the latter, it enables using Block2's `driver` and its methods to calculate the energy distribution.

The conversion itself is technically challenging, but from the user perspective can be essentially accomplished in a single line of code. We illustrate this here on the example of the CISD state.

#### Create a molecule

Start as usual by creating a simple molecule and running a Hartree-Fock calculation

In [1]:
%%capture
from pyscf import gto

from overlapper.state import do_hf

R = 2.5
N = 10
TOL = 1e-3

mol = gto.M(atom=[['H', (ii * R, 0, 0)] for ii in range(N)],
            basis='sto6g', symmetry='c1')

hf, hf_e, hf_ss, hf_sz = do_hf(mol, hftype="rhf")

#### Generate a state to be converted

As in previous notebooks, we use Overlapper to generate an initial state in the SOS sparse dictionary format -- for the sake of example, we use the CISD method

In [2]:
from overlapper.state import do_cisd, cisd_state
mycisd, mycisd_e, mycisd_ss, mycisd_sz = do_cisd(hf)
wf_cisd = cisd_state(mycisd, tol=TOL)

print("Printing the first twnety elements of the dictionary:")
for n,key in enumerate(wf_cisd.keys()):
    if n < 20:
        print(key, ":", wf_cisd[key])
        
print()        
print("The length of the whole dictionary:", len(wf_cisd))

Printing the first twnety elements of the dictionary:
(31, 31) : 0.7232248756254029
(94, 31) : 0.003184997087988679
(286, 31) : 0.009580662733417665
(61, 31) : -0.004142701529728104
(157, 31) : 0.017473369789748325
(541, 31) : 0.009298484171502613
(91, 31) : 0.023827449577377573
(283, 31) : -0.016783926655129098
(55, 31) : 0.023162604334417008
(151, 31) : 0.022757855384418782
(535, 31) : -0.0034845901635425294
(79, 31) : -0.022034353242291773
(271, 31) : -0.00440854024077261
(31, 94) : 0.003184997087988679
(31, 286) : 0.009580662733417665
(31, 61) : -0.004142701529728104
(31, 157) : 0.017473369789748325
(31, 541) : 0.009298484171502613
(31, 91) : 0.023827449577377573
(31, 283) : -0.016783926655129098

The length of the whole dictionary: 426


Notice that the wavefunction we are about to convert is large -- it has many contributing determinants.

#### Execute the conversion

Pick the number of Slater determinants from the original state to be converted and trim the state to that size, then execute the conversion in a single line of code

In [4]:
from overlapper.converter import sos_to_mps_block2
from overlapper.utils.wf_utils import wf_budget

## number of dets implemented 
n_dets = 1000
final_bond_dim = 1000
wf_cisd = wf_budget(wf_cisd, n_dets)

# execute the conversion
mps_cisd = sos_to_mps_block2(hf, wf_cisd, final_bond_dim)

Not enough Slater determinants to budget! Returning the whole thing.
Bond dimensions IN PYBLOCK3:
1|4|11|22|37|54|37|22|11|4|1
TRANSLATION TO BLOCK2 SUCCESSFUL.
ONE ROUND OF DMRG SUCCESSFUL.
Maximum bond dimension of the block2 MPS =  74
Norm of the transformed block2 MPS = 0.9999999999999997
Overlap between the converted and original wavefunctions is 1.0000.


As the built-in verification check demonstrates by calculating the overlap between the input and converted wavefunctions, the entire CISD wavefunction was perfectly converted into the MPS form. The converted wavefunction can now be used together with Overlapper methods to calculate moments or the energy distribution of the associated state.

While the example above is for a relatively small molecule, the converter functionality is well-tested and broadly applicable even for large molecules, and in cases of non-zero spin.